In [1]:
import glob
import os
from tqdm import tnrange, tqdm_notebook
from qtools import Submitter

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_bedfiles'

In [3]:
all_beds = glob.glob(os.path.join(input_dir, '*-1.bed'))
len(all_beds)

16946

In [4]:
def fix_chr(infile, outfile):
    with open(infile, 'r') as i:
        with open(outfile, 'w') as o:
            for line in i:
                o.write('chr{}'.format(line))

In [5]:
progress = tnrange(len(all_beds))
for bed in all_beds:
    output_fx_bed = os.path.splitext(bed)[0] + ".fx.bed"
    fix_chr(bed, output_fx_bed)
    progress.update(1)

In [6]:
gtfdb_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
species = 'hg19'

In [7]:
all_beds = glob.glob(os.path.join(input_dir, '*-1.fx.bed'))
len(all_beds)

16946

In [8]:
def chunker(seq, size):
    """
    Chunks a long list into groups of (size).
    """
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

groupsize = 250

cmds = []
for group in chunker(all_beds, groupsize):
    sample_str = ""
    output_str = ""
    cmd = 'module load annotator/0.0.14;annotator '
    cmd += '--gtfdb {} '.format(gtfdb_file)
    cmd += '--species {} '.format(species)
    for g in group:
        sample_str += '{} '.format(g)
        output_str += '{} '.format(g + '.annotated')
    cmd += '--input {} '.format(sample_str)
    cmd += '--output {} '.format(output_str)
    cmds.append(cmd)
    
print("Number of commands: {}".format(len(cmds)))

Number of commands: 68


In [9]:
Submitter(commands=cmds, job_name='annotate_editing_sites', array=True, nodes=1, ppn=8, submit=True, walltime='24:00:00')

Writing 68 tasks as an array-job.
Wrote commands to annotate_editing_sites.sh.
Submitted script to queue home.
 Job ID: 21215338
